Данная сеть  использует датасет созданный скурпулезно вручную и с простой сетью - исрользующую RNN. С этой сети я начал знакомиться с нейроными сетями.

In [1]:
import re

from collections import Counter

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, Input, GRU, LSTM


from keras_preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

# Работаем с датаестом

In [3]:
data=pd.read_csv("https://raw.githubusercontent.com/Terkingil/Project_NLP/main/Models%20weights%20n%20data/Name%20model%20weights/Name%20or%20abra.csv")
display(data)

,Name,Abra
0,Damir,1.0
1,Maryana,1.0
2,Roman,1.0
3,Kirill,1.0
4,Egor,1.0
...,...,...
195,Filipp,1.0
196,Yuliana,1.0
197,Veronika,1.0
198,adad,0.0


In [4]:
cdt_N=np.array(data.iloc[:,0])

Создание токенов

In [5]:
#tokenize

num_char = 63
n_tokenizer=Tokenizer(num_words=num_char, lower=False,char_level=True,filters="'!#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n'")
n_tokenizer.fit_on_texts(cdt_N)
print(n_tokenizer.word_index)

inp_chars = 3
one_hot_enc=n_tokenizer.texts_to_sequences(cdt_N)


{'a': 1, 'i': 2, 's': 3, 'o': 4, 'd': 5, 'k': 6, 'r': 7, 'l': 8, 'p': 9, 'e': 10, 'n': 11, 'A': 12, 'j': 13, 'f': 14, 'y': 15, 't': 16, 'm': 17, 'S': 18, 'q': 19, 'v': 20, 'u': 21, '[': 22, 'D': 23, '3': 24, 'M': 25, '2': 26, 'V': 27, 'z': 28, 'g': 29, '1': 30, 'w': 31, 'E': 32, ';': 33, 'h': 34, 'K': 35, 'P': 36, '4': 37, "'": 38, 'U': 39, 'N': 40, 'I': 41, 'O': 42, 'L': 43, 'B': 44, 'x': 45, '/': 46, 'T': 47, 'Y': 48, 'R': 49, 'b': 50, 'J': 51, 'F': 52, 'W': 53, 'G': 54, ' ': 55, 'H': 56, 'Z': 57, '\\': 58, 'c': 59, '9': 60}


Паддинг

In [8]:
print(one_hot_enc[2])



train_pded=pad_sequences(one_hot_enc, padding="post", truncating="post", maxlen=15)

print(train_pded[2])

[49, 4, 17, 1, 11]
[49  4 17  1 11  0  0  0  0  0  0  0  0  0  0]


Создание samples

In [9]:

X=train_pded
y=np.array(data.iloc[:,1])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21) # Выборка небольшая поэтому 10% от выборки на тест

X_train.shape,y_train.shape


((180, 15), (180,))

# Модель

In [19]:
#model

Name_model=Sequential()
Name_model.add(Embedding(63,15,input_length=15))
Name_model.add(SimpleRNN(100, activation='tanh'))
Name_model.add(Dense(1, activation='sigmoid'))#Полносвязный слой
Name_model.summary()

Name_model.compile(loss='binary_crossentropy', metrics=["AUC"])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 15, 15)            945       
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 100)               11600     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 12,646
Trainable params: 12,646
Non-trainable params: 0
_________________________________________________________________


In [20]:
history=Name_model.fit(X_train,y_train,batch_size=5,epochs=9,validation_data=(X_test,y_test))

Epoch 1/9
36/36 [==============================] - 2s 15ms/step - loss: 0.5123 - auc: 0.8247 - val_loss: 0.4882 - val_auc: 0.8535
Epoch 2/9
36/36 [==============================] - 0s 5ms/step - loss: 0.1819 - auc: 0.9781 - val_loss: 0.3252 - val_auc: 0.9394
Epoch 3/9
36/36 [==============================] - 0s 5ms/step - loss: 0.0923 - auc: 0.9963 - val_loss: 0.3796 - val_auc: 0.9192
Epoch 4/9
36/36 [==============================] - 0s 5ms/step - loss: 0.0539 - auc: 0.9983 - val_loss: 0.3815 - val_auc: 0.9495
Epoch 5/9
36/36 [==============================] - 0s 5ms/step - loss: 0.0209 - auc: 1.0000 - val_loss: 0.4743 - val_auc: 0.9141
Epoch 6/9
36/36 [==============================] - 0s 5ms/step - loss: 0.0356 - auc: 0.9995 - val_loss: 0.4433 - val_auc: 0.9242
Epoch 7/9
36/36 [==============================] - 0s 5ms/step - loss: 0.0027 - auc: 1.0000 - val_loss: 0.5572 - val_auc: 0.9242
Epoch 8/9
36/36 [==============================] - 0s 5ms/step - loss: 9.0473e-04 - auc: 1.0000 

In [23]:
#Name_model.save_weights('./N_model/N_weights')

# Проверка ручным вводом

In [26]:
a="Andrey"
inp=[a]

inp_sname_tkzd=n_tokenizer.texts_to_sequences(inp)
#print(inp_enc)
inp_sname_tkzd_pded=pad_sequences(inp_sname_tkzd, padding="post", truncating="post", maxlen=15)

  #print(inp_enc_pded)

surname_hyppo = (Name_model.predict(inp_sname_tkzd_pded))

surname_hyppo

1/1 [==============================] - 0s 171ms/step


array([[0.9999654]], dtype=float32)